In [1]:
import os
import datetime
import json
import logging
import warnings
import time
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from email import encoders
import smtplib
from io import BytesIO
import pandas as pd

from reportlab.pdfgen import canvas
from reportlab.lib.units import mm
import barcode
from barcode.writer import ImageWriter
from reportlab.lib.utils import ImageReader

import requests
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta

# import azure.functions as func
from azure.identity import DefaultAzureCredential
from azure.keyvault.secrets import SecretClient

# app = func.FunctionApp()

key_vault_url = "https://premo-vault.vault.azure.net/"
credential = DefaultAzureCredential()
client = SecretClient(vault_url=key_vault_url, credential=credential)

In [2]:
DI_USERNAME = client.get_secret('DI-USERNAME').value
DI_PASSWORD = client.get_secret('DI-PASSWORD').value
SENDER_EMAIL = client.get_secret("GMAIL-USERNAME").value
SENDER_PASSWORD = client.get_secret("GMAIL-APP-PASS").value
BRANCH_EMAILS = json.loads(client.get_secret('BRANCH-EMAILS').value)
_TO_EMAILS_DEV = os.getenv('_TO_EMAILS_DEV', '')
_TO_EMAILS_PROD = os.getenv('_TO_EMAILS_PROD', '')

In [3]:
def create_di_session(DI_USERNAME, DI_PASSWORD):
    session = requests.Session()
    login_url = "https://app.di.no/app/api/login"
    payload = {
        'username': DI_USERNAME,
        'password': DI_PASSWORD
    }

    response = session.post(login_url, data=payload)

    if response.status_code == 200 and "topnavigation" in response.text:
        logging.info("Login successful")
        return session
    else:
        raise Exception("Login failed")

In [4]:
def fetch_di_report(session, report_url, **kwargs):
    """
    Fetches a DI report from the given URL using the provided session and returns it as a pandas DataFrame.

    Parameters:
        session (requests.Session): The session object used to make the HTTP GET request.
        report_url (str): The URL of the report to fetch.
        **kwargs: Optional keyword arguments to pass to pd.read_excel, such as dtype.

    Returns:
        pd.DataFrame: The report data as a pandas DataFrame.

    Raises:
        Exception: If the HTTP request fails (status code not 200).
    """
    try:
        report_response = session.get(report_url)
        if report_response.status_code == 200:
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                df = pd.read_excel(
                    BytesIO(report_response.content),
                    engine='openpyxl',
                    **kwargs  # Pass any additional keyword arguments here
                )
            logging.info(f"Report fetched and parsed successfully from {report_url}")
            return df
        else:
            raise Exception(f"Failed to download the file. Status code: {report_response.status_code} for URL: {report_url}")
    except Exception as e:
        logging.error(f"An error occurred while fetching the report: {e}")
        raise

In [5]:
def send_email_with_pdf_from_df(
    sender_email: str,
    sender_password: str,
    to_emails: str,
    cc_emails: str,
    subject: str,
    body: str,
    pdf_bytes: BytesIO = None,
    pdf_filename: str = None,
    max_retries: int = 3
):
    """
    Sends an email with an optional PDF attachment.
    Args:
        sender_email (str): Sender's email address.
        sender_password (str): Sender's email password.
        to_emails (str): Comma-separated recipient email addresses.
        cc_emails (str): Comma-separated CC email addresses.
        subject (str): Subject of the email.
        body (str): Body of the email in HTML format.
        pdf_bytes (BytesIO, optional): In-memory bytes of the PDF file. Defaults to None.
        pdf_filename (str, optional): Filename for the PDF attachment. Defaults to None.
        max_retries (int, optional): Maximum number of retry attempts. Defaults to 3.
    """
    smtp_server = 'smtp.gmail.com'
    smtp_port = 587

    # Create the email message
    message = MIMEMultipart()
    message['From'] = sender_email
    message['To'] = to_emails
    message['Cc'] = cc_emails
    message['Subject'] = subject

    # Attach the email body
    message.attach(MIMEText(body, 'html'))

    # Attach the PDF file if provided
    if pdf_bytes:
        try:
            pdf_bytes.seek(0)
            attachment = MIMEApplication(pdf_bytes.read(), _subtype='pdf')
            attachment_filename = pdf_filename or 'attachment.pdf'  # Determine the filename
            attachment.add_header(
                'Content-Disposition',
                'attachment',
                filename=attachment_filename
            )
            message.attach(attachment)
            logging.info(f"Attached PDF '{pdf_filename}' successfully.")
        except Exception as e:
            logging.error(f"Failed to attach PDF '{pdf_filename}': {e}")
            message.attach(MIMEText('<br><strong>** FAILED TO ATTACH PDF **</strong>', 'html'))

    # Combine recipients
    recipients = to_emails.split(',') + cc_emails.split(',')

    # Attempt to send the email with retries
    for attempt in range(1, max_retries + 1):
        try:
            with smtplib.SMTP(smtp_server, smtp_port) as server:
                server.starttls()
                server.login(sender_email, sender_password)
                server.send_message(message, from_addr=sender_email, to_addrs=recipients)
            logging.info(f"Email sent to {', '.join(recipients)} at {datetime.datetime.now(pytz.timezone('Europe/Stockholm')).strftime('%Y-%m-%d %H:%M')}")
            break
        except Exception as e:
            logging.error(f'Attempt {attempt} failed: {e}')
            if attempt < max_retries:
                logging.info('Retrying in 5 seconds...')
                time.sleep(5)
            else:
                logging.error(f'Failed to send email after {max_retries} attempts')

In [6]:
## CHANGE REPORT DATE FOR PROD ##
report_date = datetime.datetime.now(pytz.timezone('Europe/Stockholm')).date() + relativedelta(days=0)
report_date_di_format = datetime.datetime.strftime(report_date, '%d.%m.%Y')
report_url = (f"https://app.di.no/app/ReportGenerator.do?action=getReport&"
            f"selectedReport=291&parameterValue=&index=&token=1724976235436&"
            f"parameterSelectedValues=519&parameterMultiSelectedValues=&"
            f"parameterSelectedValues=7540&parameterMultiSelectedValues=&"
            f"parameterSelectedValues={report_date_di_format}&parameterMultiSelectedValues=&"
            f"parameterSelectedValues=false&parameterMultiSelectedValues=&"
            f"reportFormatAsString=XLSX&reportEncodingAsString=ISO_8859_1")
report_dtype = {'KundeId': str}

In [7]:
session = create_di_session(DI_USERNAME, DI_PASSWORD)
raw_parcel_df = fetch_di_report(session, report_url, dtype=report_dtype)

In [20]:
return_parcel_df = raw_parcel_df[raw_parcel_df['Returkode'].notna()]
return_parcel_sorted_df = return_parcel_df.sort_values(by=['Region', 'Omraade', 'Returkode'])
return_parcel_sorted_df.head()

,Produkt,Dager,Bedrift,Region,Omraade,Start,Stopp,Distrikt,KundeId,Navn,...,Hoyde,Vekt,FÃ¸rste visning,OkSkannDato,OkSkannStasjon,Aktiv,KundeSysNavn,EksternengangsleveranseId,Sporingsnummer,Returkode
457,RETUR,NaN,Premo,Sthlm City,ESSINGEN,2024-11-28,NaN,8017206,00973402216087356454,Sofie Pettersson,...,0,500.0,JA,NaN,NaN,1,Vinted,163871054.0,973402216087356454,E-634019
835,RETUR,NaN,Premo,Sthlm City,ESSINGEN,2024-11-28,NaN,8017224,00673402216025779863,Evelina Lindberg,...,0,500.0,JA,NaN,NaN,1,Vinted,164067177.0,673402216025779863,E-638303
566,RETUR,NaN,Premo,Sthlm City,ESSINGEN,2024-11-28,NaN,8017207,00773402216013251309,Clara Boulund,...,0,2000.0,JA,NaN,NaN,1,Vinted,164125114.0,773402216013251309,E-640940
729,RETUR,NaN,Premo,Sthlm City,ESSINGEN,2024-11-28,NaN,8017210,00573402216054745955,Hanna Korell,...,0,500.0,JA,NaN,NaN,1,Vinted,164178698.0,573402216054745955,E-641920
394,RETUR,NaN,Premo,Sthlm City,ESSINGEN,2024-11-28,NaN,8017205,00473402216083931834,Kristof Vogel,...,0,2000.0,JA,NaN,NaN,1,Vinted,164186159.0,473402216083931834,E-642051


In [21]:
## DEBUG, DON't USE IN PRODUCTION ##
debug_return_parcel_sorted_df = return_parcel_sorted_df[return_parcel_sorted_df['Omraade'].isin(['ESSINGEN', 'MATTEUS', 'JAKOB', 'HANINGE'])]
debug_return_parcel_sorted_df

,Produkt,Dager,Bedrift,Region,Omraade,Start,Stopp,Distrikt,KundeId,Navn,...,Hoyde,Vekt,FÃ¸rste visning,OkSkannDato,OkSkannStasjon,Aktiv,KundeSysNavn,EksternengangsleveranseId,Sporingsnummer,Returkode
457,RETUR,NaN,Premo,Sthlm City,ESSINGEN,2024-11-28,NaN,8017206,00973402216087356454,Sofie Pettersson,...,0,500.0,JA,NaN,NaN,1,Vinted,163871054.0,973402216087356454,E-634019
835,RETUR,NaN,Premo,Sthlm City,ESSINGEN,2024-11-28,NaN,8017224,00673402216025779863,Evelina Lindberg,...,0,500.0,JA,NaN,NaN,1,Vinted,164067177.0,673402216025779863,E-638303
566,RETUR,NaN,Premo,Sthlm City,ESSINGEN,2024-11-28,NaN,8017207,00773402216013251309,Clara Boulund,...,0,2000.0,JA,NaN,NaN,1,Vinted,164125114.0,773402216013251309,E-640940
729,RETUR,NaN,Premo,Sthlm City,ESSINGEN,2024-11-28,NaN,8017210,00573402216054745955,Hanna Korell,...,0,500.0,JA,NaN,NaN,1,Vinted,164178698.0,573402216054745955,E-641920
394,RETUR,NaN,Premo,Sthlm City,ESSINGEN,2024-11-28,NaN,8017205,00473402216083931834,Kristof Vogel,...,0,2000.0,JA,NaN,NaN,1,Vinted,164186159.0,473402216083931834,E-642051
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17658,RETUR,NaN,Premo,Sthlm Söder,HANINGE,2024-11-28,NaN,8136016,00873402216018053950,Renata Krol,...,0,500.0,JA,NaN,NaN,1,Vinted,164370103.0,873402216018053950,E-647050
16903,RETUR,NaN,Premo,Sthlm Söder,HANINGE,2024-11-28,NaN,8136006,00573402216059021580,Reda Aleksandraviciene,...,0,500.0,JA,NaN,NaN,1,Vinted,164371169.0,573402216059021580,E-647102
17892,RETUR,NaN,Premo,Sthlm Söder,HANINGE,2024-11-28,NaN,8136019,00473402216040093636,Anneli Olsson,...,0,500.0,JA,NaN,NaN,1,Vinted,164374427.0,473402216040093636,E-647276
16649,RETUR,NaN,Premo,Sthlm Söder,HANINGE,2024-11-28,NaN,8136001,00573402216049224809,Ida Nurhonen,...,0,500.0,JA,NaN,NaN,1,Vinted,164374997.0,573402216049224809,E-647315


In [ ]:
## GENERATE LABELS WITHOUT SEPARATOR ##

In [22]:
from io import BytesIO
from reportlab.lib.units import mm
from reportlab.pdfgen import canvas
from reportlab.lib.utils import ImageReader
import barcode
from barcode.writer import ImageWriter

def create_label(c, row, page_width, page_height):
    barcode_value = str(row['Returkode'])
    code128 = barcode.get('code128', barcode_value, writer=ImageWriter())
    barcode_image = code128.render(writer_options={'write_text': False})

    # Create an ImageReader from the PIL Image
    image_reader = ImageReader(barcode_image)

    # Margins and spacing
    top_margin = 10 * mm
    left_margin = 10 * mm
    line_height = 8 * mm

    # Draw the label content 
    c.setFont("Helvetica", 12)
    c.drawString(
        left_margin,
        page_height - top_margin - 0 * line_height,
        f"Region: {row['Region']}"
    )
    c.drawString(
        left_margin,
        page_height - top_margin - 1 * line_height,
        f"Filial: {row['Omraade']}"
    )
    c.drawString(
        left_margin,
        page_height - top_margin - 2 * line_height,
        f"Trakt: {row['Distrikt']}"
    )

    # Returkode text
    c.setFont("Helvetica-Bold", 24)
    returkode_text = str(row['Returkode'])
    text_width = c.stringWidth(returkode_text, "Helvetica-Bold", 24)
    x_position = (page_width - text_width) / 2
    c.drawString(x_position, 35 * mm, returkode_text)

    # Draw the barcode image
    barcode_width = page_width - 2 * left_margin
    barcode_height = 20 * mm
    x_image_position = left_margin
    y_image_position = 10 * mm
    c.drawImage(
        image_reader,
        x_image_position,
        y_image_position,
        width=barcode_width,
        height=barcode_height
    )

def generate_labels_pdf(df):
    pdf_buffer = BytesIO()
    page_width = 102 * mm
    page_height = 76 * mm
    c = canvas.Canvas(pdf_buffer, pagesize=(page_width, page_height))

    for index, row in df.iterrows():
        create_label(c, row, page_width, page_height)
        c.showPage()

    c.save()
    pdf_buffer.seek(0)
    return pdf_buffer


In [23]:
pdf_label_buffer = generate_labels_pdf(debug_return_parcel_sorted_df)

In [ ]:
send_email_with_pdf_from_df(
    sender_email=SENDER_EMAIL,
    sender_password=SENDER_PASSWORD,
    to_emails='johan.tokarskij@premo.se',
    cc_emails='',
    subject='Barcodes för upphämtade paket',
    body='',
    pdf_bytes=pdf_label_buffer,
    pdf_filename='labels.pdf'
)

In [28]:
## GENERATE LABELS WITH SIMGLE SEPARATOR ##

In [33]:
from io import BytesIO
from reportlab.lib.units import mm
from reportlab.pdfgen import canvas
from reportlab.lib.utils import ImageReader
import barcode
from barcode.writer import ImageWriter
import pandas as pd

def create_label(c, row, page_width, page_height):
    barcode_value = str(row['Returkode'])
    code128 = barcode.get('code128', barcode_value, writer=ImageWriter())
    barcode_image = code128.render(writer_options={'write_text': False})

    image_reader = ImageReader(barcode_image)

    top_margin = 10 * mm
    left_margin = 10 * mm
    line_height = 8 * mm

    # Draw the label content
    c.setFont("Helvetica", 12)
    c.drawString(left_margin, page_height - top_margin, f"Region: {row['Region']}")
    c.drawString(left_margin, page_height - top_margin - line_height, f"Filial: {row['Omraade']}")
    c.drawString(left_margin, page_height - top_margin - 2 * line_height, f"Trakt: {row['Distrikt']}")

    # Returkode text
    c.setFont("Helvetica-Bold", 24)
    returkode_text = str(row['Returkode'])
    text_width = c.stringWidth(returkode_text, "Helvetica-Bold", 24)
    x_position = (page_width - text_width) / 2
    c.drawString(x_position, 35 * mm, returkode_text)

    # Draw the barcode image
    barcode_width = page_width - 2 * left_margin
    barcode_height = 20 * mm
    c.drawImage(image_reader, left_margin, 10 * mm, width=barcode_width, height=barcode_height)

def create_separator(c, omraade, page_width, page_height):
    c.setFont("Helvetica-Bold", 36)
    text = omraade
    text_width = c.stringWidth(text, "Helvetica-Bold", 36)
    x_position = (page_width - text_width) / 2
    y_position = (page_height - 36) / 2
    c.drawString(x_position, y_position, text)

def generate_labels_pdf_with_single_separator(df):
    # Sort the DataFrame to group similar Omraade together
    df_sorted = df.sort_values(by=['Region', 'Omraade', 'Returkode']).reset_index(drop=True)

    pdf_buffer = BytesIO()
    page_width = 102 * mm
    page_height = 76 * mm
    c = canvas.Canvas(pdf_buffer, pagesize=(page_width, page_height))

    previous_omraade = None

    for index, row in df_sorted.iterrows():
        current_omraade = row['Omraade']
        
        # Insert a separator label whenever Omraade changes
        if current_omraade != previous_omraade:
            create_separator(c, current_omraade, page_width, page_height)
            c.showPage()
            previous_omraade = current_omraade

        # Create the standard label
        create_label(c, row, page_width, page_height)
        c.showPage()

    c.save()
    pdf_buffer.seek(0)
    return pdf_buffer

In [ ]:
pdf_label_buffer_with_single_separator = generate_labels_pdf_with_single_separator(debug_return_parcel_sorted_df)

In [ ]:
send_email_with_pdf_from_df(
    sender_email=SENDER_EMAIL,
    sender_password=SENDER_PASSWORD,
    to_emails='johan.tokarskij@premo.se',
    cc_emails='',
    subject='Barcodes för upphämtade paket',
    body='',
    pdf_bytes=pdf_label_buffer_with_single_separator,
    pdf_filename='labels_with_single_separator.pdf'
)

In [36]:
## GENERATE LABELS WITH DOUBLE SEPARATOR ##

In [38]:
def create_label(c, row, page_width, page_height):
    barcode_value = str(row['Returkode'])
    code128 = barcode.get('code128', barcode_value, writer=ImageWriter())
    barcode_image = code128.render(writer_options={'write_text': False})

    image_reader = ImageReader(barcode_image)

    top_margin = 10 * mm
    left_margin = 10 * mm
    line_height = 8 * mm

    # Draw the label content
    c.setFont("Helvetica", 12)
    c.drawString(left_margin, page_height - top_margin, f"Region: {row['Region']}")
    c.drawString(left_margin, page_height - top_margin - line_height, f"Filial: {row['Omraade']}")
    c.drawString(left_margin, page_height - top_margin - 2 * line_height, f"Trakt: {row['Distrikt']}")

    # Returkode text
    c.setFont("Helvetica-Bold", 24)
    returkode_text = str(row['Returkode'])
    text_width = c.stringWidth(returkode_text, "Helvetica-Bold", 24)
    x_position = (page_width - text_width) / 2
    c.drawString(x_position, 35 * mm, returkode_text)

    # Draw the barcode image
    barcode_width = page_width - 2 * left_margin
    barcode_height = 20 * mm
    c.drawImage(image_reader, left_margin, 10 * mm, width=barcode_width, height=barcode_height)

def create_separator(c, omraade, page_width, page_height, status):
    # Main Omraade text
    c.setFont("Helvetica-Bold", 36)
    text_width = c.stringWidth(omraade, "Helvetica-Bold", 36)
    x_position = (page_width - text_width) / 2
    y_position = (page_height - 36) / 2 + 10  # Adjusted to accommodate status text
    c.drawString(x_position, y_position, omraade)

    # Status text ("Start" or "Stop")
    if status in ["Start", "Stop"]:
        c.setFont("Helvetica", 18)
        status_text = status
        status_width = c.stringWidth(status_text, "Helvetica", 18)
        status_x = (page_width - status_width) / 2
        status_y = y_position - 20  # Position below the Omraade text
        c.drawString(status_x, status_y, status_text)

def generate_labels_pdf_with_double_separators(df):
    # Sort the DataFrame to group similar Omraade together
    df_sorted = df.sort_values(by=['Region', 'Omraade', 'Returkode']).reset_index(drop=True)

    pdf_buffer = BytesIO()
    page_width = 102 * mm
    page_height = 76 * mm
    c = canvas.Canvas(pdf_buffer, pagesize=(page_width, page_height))

    for index in range(len(df_sorted)):
        row = df_sorted.iloc[index]
        current_omraade = row['Omraade']

        # Determine if this row is the first in its Omraade group
        if index == 0 or current_omraade != df_sorted.iloc[index - 1]['Omraade']:
            create_separator(c, current_omraade, page_width, page_height, status="Start")
            c.showPage()

        # Create the standard label
        create_label(c, row, page_width, page_height)
        c.showPage()

        # Determine if this row is the last in its Omraade group
        is_last_in_group = (index == len(df_sorted) - 1) or (current_omraade != df_sorted.iloc[index + 1]['Omraade'])
        if is_last_in_group:
            create_separator(c, current_omraade, page_width, page_height, status="Stop")
            c.showPage()

    c.save()
    pdf_buffer.seek(0)
    return pdf_buffer

In [39]:
pdf_label_buffer_with_double_separator = generate_labels_pdf_with_double_separators(debug_return_parcel_sorted_df)

In [ ]:
send_email_with_pdf_from_df(
    sender_email=SENDER_EMAIL,
    sender_password=SENDER_PASSWORD,
    to_emails='johan.tokarskij@premo.se',
    cc_emails='',
    subject='Barcodes för upphämtade paket',
    body='',
    pdf_bytes=pdf_label_buffer_with_double_separator,
    pdf_filename='labels_with_double_separators.pdf'
)